<a href="https://colab.research.google.com/github/thevineetsangwan/Dev/blob/master/Yatra_GenAI_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install google-cloud-aiplatform --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.1 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
! pip show google-cloud-aiplatform

Name: google-cloud-aiplatform
Version: 1.28.1
Summary: Vertex AI API client library
Home-page: https://github.com/googleapis/python-aiplatform
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /root/.local/lib/python3.10/site-packages
Requires: google-api-core, google-cloud-bigquery, google-cloud-resource-manager, google-cloud-storage, packaging, proto-plus, protobuf, shapely
Required-by: 


In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
import vertexai
from vertexai.preview.language_models import ChatModel, InputOutputTextPair


In [ ]:
vertexai.init(project="mlconsole-poc", location="us-central1")
chat_model = ChatModel.from_pretrained("chat-bison@001")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 1024,
    "top_p": 1,
    "top_k": 40
}

In [ ]:
def get_prompt():
  prompt = """Following is the data of a company\'s employees travel history some employees have booked tickets for round way trip, while some have booked for one-way trip :
  {
      {
    \"Corporate\": \"BCG\",
    \"Employee\": \"Ravi Shankar\",
    \"From\": \"DEL\",
    \"To\": \"BOM\",
    \"Departure Date\": \"July 1, 2023\",
    \"Return Date\": \"July 3, 2023\",
    \"Flight Outbound\": \"6E5112\",
    \"Flight Inbound\": \"6E2139\"
  }
  {
    \"Corporate\": \"BCG\",
    \"Employee\": \"Shakti Goel\",
    \"From\": \"DEL\",
    \"To\": \"BLR\",
    \"Departure Date\": \"July 4, 2023\",
    \"Flight Outbound\": \"6E308\"
  }
  {
    \"Corporate\": \"BCG\",
    \"Employee\": \"Ravi Shankar\",
    \"From\": \"BOM\",
    \"To\": \"MAA\",
    \"Departure Date\": \"June 5, 2023\",
    \"Flight Inbound\": \"UK839\"
  }
  {
    \"Corporate\": \"BCG\",
    \"Employee\": \"Ravi Shankar\",
    \"From\": \"BOM\",
    \"To\": \"MAA\",
    \"Departure Date\": \"May 15. 2022\",
    \"Return Date\": \"June 5, 2023\",
    \"Flight Outbound\": \"AI429\",
    \"Flight Inbound\": \"6E6276\"
  }
  }

  system : You are an assistant who will help users with their queries regarding flight details or ticket booking.
  If user want to take same flight as their colleague then remember the flight number and book the that flight for specified date.
  Answer according to what user asks, user can ask about the following :
  1. User can ask you to book a flight, user can specify departure date in query or user can tell to book ticket for date after few days from today. Today\'s date is 25th July 2023 and today date is same as now or current date.

  Here are few examples:
  user : book a flight for 28th July.
  departure date : 28th July 2023

  user : book flight for 2 weeks from now. (1 week is same as 7 days)
  departure date : 8th August 2023

  user : book flight for 3 weeks from now.
  departure date : 15th August 2023

  user : book flight for 3 weeks from tomorrow.
  departure date : 16th August 2023

  user : book a flight for 8 days from today
  departure date : 2nd August 2023

  Answer this query in JSON format, JSON should contain the entities Employee name, Departure station, travel date, flight number.

  2. User can ask for the details for a specific flight or user can ask for details for more than one flight so you have to answer this in text format.
  """
  return prompt

def get_examples():
  examples=[
          InputOutputTextPair(
              input_text="""My name is Ravi Shankar.  Make a booking for 2 weeks from now from Mumbai to Chennai, I want to take the flights that I took on June 5, 2023.""",
              output_text="""{
    \"Corporate\": \"BCG\",
    \"Employee\": \"Ravi Shankar\",
    \"From\": \"BOM\",
    \"To\": \"MAA\",
    \"Departure Date\": \"August 8, 2023\",
    \"Flight Outbound\": \"UK839\"
  }
  """
          ),
          InputOutputTextPair(
              input_text="""My name is Ravi Shankar.  Make a booking for 5 days from now from Mumbai to Chennai, I want to take the flights that I took on June 5, 2023.""",
              output_text="""{
    \"Corporate\": \"BCG\",
    \"Employee\": \"Ravi Shankar\",
    \"From\": \"BOM\",
    \"To\": \"MAA\",
    \"Departure Date\": \"July 29, 2023\",
    \"Flight Outbound\": \"UK839\"
  }"""
  )
  ]
  return examples

In [ ]:
import requests

def flights_api(data):
  url = "https://yatra-flight-api-njsl6nofca-uc.a.run.app"
  r = requests.post(url=url, json=data)
  print(r.json())

In [ ]:
def get_prediction(query, book):
  prompt = get_prompt()
  examples = get_examples()

  chat = chat_model.start_chat(
    context=prompt,
    examples = examples
  )

  # This is an Custom API Call to get the price of the flight being discussed
  response = chat.send_message(query, **parameters)
  if book == True:
    data = eval(f"{response.text}")
    flights_api(data)
  elif book == False:
    print(f"{response.text}")

In [ ]:
get_prediction("My name is Ravi Shankar.  Make a booking for Bangalore from Delhi two weeks from now.  I want to take the flights that Shakti took.", True)

{'Corporate': 'BCG', 'Departure Date': 'August 8, 2023', 'Employee': 'Ravi Shankar', 'Flight Outbound': '6E308', 'From': 'DEL', 'To': 'BLR', 'price': 5236}


In [ ]:
get_prediction("My name is Ravi Shankar. Book the same flight I took last time to go to Chennai from Bombay.  Book it for 8 days from now.", True)

{'Corporate': 'BCG', 'Departure Date': 'August 2, 2023', 'Employee': 'Ravi Shankar', 'Flight Outbound': 'UK839', 'From': 'BOM', 'To': 'MAA', 'price': 4000}


In [ ]:
get_prediction("I am Ravi Shankar.  What was the flight that my colleague Shakti took last?", False)

On July 4, 2023, Shakti Goel took flight 6E308 from Delhi to Bangalore.


In [ ]:
get_prediction("List all the round trip flights that Ravi Shankar took in last 4 months.", False)

Ravi Shankar took following round trip flights in last 4 months

1. May 15. 2022, BOM to MAA, AI429, 6E6276
2. July 1, 2023, DEL to BOM, 6E5112, 6E2139


In [ ]:
get_prediction("tell me about the second flight", False)

Ravi Shankar's flight from Mumbai to Chennai on June 5, 2023 had flight number UK839.


In [ ]:
get_prediction("tell me about the first flight", False)

The first flight is from Delhi to Mumbai on July 1, 2023. The employee name is Ravi Shankar and the flight number is 6E5112.


In [ ]:
get_prediction("list all the travellers who have travelled this year", False)

The following employees have travelled this year:
Shakti Goel
Ravi Shankar


In [ ]:
get_prediction("I am Nitin, book me same flight which Ravi Shankar took on 1st July", True)

{'Corporate': 'BCG', 'Departure Date': 'July 1, 2023', 'Employee': 'Nitin', 'Flight Outbound': '6E5112', 'From': 'DEL', 'To': 'BOM', 'price': 5112}
